In [1]:
import AcTOL
import torch
from PIL import Image
# Load AcTOL model

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AcTOL.load("AcTOL", device=device)

image = Image.open("Your Image Path Here")
text = "Your Instruction Here"

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    reward = model.get_reward(image, text)

/home/xli990/miniconda3/envs/AcTOL/lib/python3.10/site-packages/clip/clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging
/home/xli990/miniconda3/envs/AcTOL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xli990/miniconda3/envs/AcTOL/lib/python3.10/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0dev0)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy,

===========Loading AcTOL Model==============


100%|███████████████████████████████████████| 244M/244M [00:02<00:00, 85.6MiB/s]
/home/xli990/miniconda3/envs/AcTOL/lib/python3.10/site-packages/torch/cuda/__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5090 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75 sm_80 sm_86.
If you want to use the NVIDIA GeForce RTX 5090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


UnpicklingError: invalid load key, '<'.

In [ ]:
import os, numpy as np, torch
import matplotlib.pyplot as plt
from PIL import Image
import AcTOL

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AcTOL.load("AcTOL", device=device)

def get_reward_curve(DATA_DIR, participant, video_id, start_f, end_f, instruction, num_points=80):
    idxs = np.linspace(start_f, end_f, num_points).round().astype(int)
    rewards = []
    kept = []

    for i in idxs:
        p = os.path.join(DATA_DIR, participant, "rgb_frames", video_id, f"frame_{i:010d}.jpg")
        if not os.path.exists(p):
            continue
        img = Image.open(p).convert("RGB")
        r = model.get_reward(img, instruction)
        rewards.append(float(r.detach().cpu().item() if torch.is_tensor(r) else r))
        kept.append(i)

    return np.array(kept), np.array(rewards, dtype=np.float32)

def plot_curve(rewards, title=""):
    r = (rewards - rewards.min()) / (rewards.max() - rewards.min() + 1e-8)  # normalize 0-1
    plt.figure(figsize=(9,3.5))
    plt.plot(r, linewidth=2.3)
    plt.ylim(0,1.05)
    plt.xlabel("Sampled frame step")
    plt.ylabel("Normalized reward")
    plt.title(title)
    plt.grid(alpha=0.25)
    plt.show()


In [ ]:
DATA_DIR = r"G:/EPIC-KITCHEN-100/2g1n6qdydwa9u22shpxqzp0t8m"
participant = "P01"
video_id = "P01_109"
start_frame, end_frame = 3719, 3799
instruction = "open cupboard"

frame_ids, rewards = get_reward_curve(DATA_DIR, participant, video_id, start_frame, end_frame, instruction, num_points=80)
plot_curve(rewards, title=f"{participant}-{video_id} | {instruction}")
